# Importation des données

In [13]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import chi2_contingency

customers = pd.read_csv('data/customers.csv')
products = pd.read_csv('data/products.csv')
transactions = pd.read_csv('data/transactions.csv')

# Obtenir les infos en fonction de conditions

In [ ]:
products.loc[(products["id_prod"] == "0_1507"), ["id_prod", "price", "categ"]]

# Vérification si il y a des transactions nulles

In [ ]:
missing_values = transactions.isnull().sum()
missing_values

# Vérification des doublons

In [ ]:
duplicate_transac = transactions[transactions.duplicated()]
duplicate_transac

In [ ]:
duplicate_prod = products[products.duplicated()]
duplicate_prod

In [ ]:
duplicate_custom = customers[customers.duplicated()]
duplicate_custom

# Ratio Hommes / Femmes

In [ ]:
gender_counts = customers['sex'].value_counts()
plt.figure(figsize=(7, 6))
ax = gender_counts.plot(kind='bar', rot=0, color=["tab:pink", "tab:blue"])
ax.set_title("Ratio Femmes / Hommes", y=1)
ax.set_xlabel('Genre')
ax.set_ylabel('Nombre')
ax.set_xticklabels(('Femme', 'Homme'))
for rect in ax.patches:
    y_value = rect.get_height()
    x_value = rect.get_x() + rect.get_width() / 2
    space = 1
    label = format(y_value)
    ax.annotate(label, (x_value, y_value), xytext=(0, space), textcoords="offset points", ha='center', va='bottom')
plt.show()

# Nombre de clients par année de naissance en fonction du genre

In [ ]:
customers_by_sex_birth = customers.groupby(['sex', 'birth']).size().reset_index(name='count')
pivot_customers = customers_by_sex_birth.pivot(index='birth', columns='sex', values='count').fillna(0)
pivot_customers['Total'] = pivot_customers.sum(axis=1)
pivot_customers_sorted = pivot_customers.sort_values(by='Total')
sns.set(style="whitegrid")
plt.figure(figsize=(15, 6))
pivot_customers[['f', 'm']].plot(kind='bar', stacked=True, color=['#ff9999', '#9999ff'], figsize=(15, 6))
plt.title('Nombre de Clients par Année de Naissance avec Distinction des Sexes')
plt.xlabel('Année de Naissance')
plt.ylabel('Nombre de Clients')
plt.xticks(rotation=45)
plt.legend(title='Sexe', labels=['Féminin', 'Masculin'])
plt.tight_layout()
plt.show()

# Tableau croisé entre sex et categorie

In [ ]:
tableau_croise = pd.crosstab(customers['sex'], products['categ'])
tableau_croise

# Les prix par catégorie

In [ ]:
price_by_category = products.groupby('categ')['price'].describe()

price_by_category

# Retrait du produit négatif

In [ ]:
products = products[products['price'] != -1]

produits_negatif_restant = products[products['price'] < 0].shape[0]

produits_negatif_restant

# Visualisation des prix par catégorie

In [ ]:
plt.figure(figsize=(10, 6))
sns.boxplot(x='categ', y='price', data=products)

plt.title('Distribution des prix par catégorie de produits')
plt.xlabel('Catégorie')
plt.ylabel('Prix')
plt.xticks([0, 1, 2], ['Catégorie 0', 'Catégorie 1', 'Catégorie 2'])

plt.show()

# Nettoyage des transactions avec les dates test

In [ ]:
transactions_cleaned = transactions[~transactions['date'].str.startswith("test")]
transactions_with_test = transactions[transactions['date'].str.startswith("test")]

original_count = transactions.shape[0]
cleaned_count = transactions_cleaned.shape[0]
test_transactions_removed = original_count - cleaned_count

(original_count, cleaned_count, test_transactions_removed, transactions_cleaned, transactions_with_test)

# Produits les plus vendus

In [ ]:
transactions_by_product = transactions_cleaned.groupby('id_prod').size().sort_values(ascending=False)

transactions_by_product.head()

# Prix des articles

In [ ]:
freq_achat = products.groupby('price').agg({'id_prod': ', '.join})
freq_achat

# CA par produit

In [ ]:
transactions_with_price = transactions_cleaned.merge(products[['id_prod', 'price']], on='id_prod', how='left')

total_revenue_by_product = transactions_with_price.groupby('id_prod').agg(
    transactions_count=('id_prod', 'size'),
    total_revenue=('price', 'sum')
).sort_values(by='total_revenue', ascending=False)

total_revenue_by_product.head()

# Graphique du top 10 des produits avec le plus gros CA

In [ ]:
top_10_CA_products = total_revenue_by_product.head(10).reset_index()

plt.figure(figsize=(14, 8))
sns.barplot(x='total_revenue', y='id_prod', data=top_10_CA_products, palette='coolwarm_r') #_r permet d'inverser les couleurs

plt.title('Top 10 des produits générant le plus de revenus')
plt.xlabel('Revenu Total')
plt.ylabel('Identifiant du Produit')
plt.show()

# Produits jamais vendus

In [45]:
sold_products = transactions_cleaned['id_prod'].unique()
unsold_products = products[~products['id_prod'].isin(sold_products)]

unsold_products

unsold_products_df = products[~products['id_prod'].isin(sold_products)]
chemin_fichier_csv_unsold = 'data/unsold_products.csv'
unsold_products_df.to_csv(chemin_fichier_csv_unsold, index=False)

In [ ]:
jointure_outer = pd.merge(transactions, products, on='id_prod', how='outer')
non_vendus = jointure_outer[jointure_outer['session_id'].isna()]
non_vendus_final =non_vendus["id_prod"].count()
print("Nombre de produits non vendus :", non_vendus_final)

# Nombre de transactions par date

In [ ]:
transactions['date_converted'] = pd.to_datetime(transactions['date'], errors='coerce')

data_filtered = transactions.dropna(subset=['date_converted'])

data_filtered['just_date'] = data_filtered['date_converted'].dt.date

transactions_by_date_filtered = data_filtered.groupby('just_date').size()

plt.figure(figsize=(12, 6))
transactions_by_date_filtered.plot(kind='line', color='blue', linewidth=2)
plt.title('Nombre de transactions par date')
plt.xlabel('Date')
plt.ylabel('Nombre de transactions')
plt.tight_layout()
plt.show()

# Détail du nombre de transactions sur la période suspecte

In [ ]:
transactions_cleaned['date'] = pd.to_datetime(transactions_cleaned['date'], errors='coerce')

transactions_month = transactions_cleaned[transactions_cleaned['date'].dt.month.isin([8, 9, 10, 11, 12])]

transactions_month_with_categ = transactions_month.merge(products[['id_prod', 'price', 'categ']], on='id_prod', how='left')

transactions_details_by_categ = transactions_month_with_categ.groupby([transactions_month_with_categ['date'].dt.month, 'categ']).agg(
    number_of_transactions=('id_prod', 'count'),
    total_revenue=('price', 'sum')
).rename(index={8: 'Aout', 9: 'September', 10: 'October', 11: 'November', 12: 'December'}, level=0)

transactions_details_by_categ

# Nombre de transactions par jour en Octobre

In [ ]:
transactions_with_categ = transactions_cleaned.merge(products[['id_prod', 'price', 'categ']], on='id_prod', how='left')

transactions_oct_categ1 = transactions_with_categ[
    (transactions_with_categ['date'].dt.month == 10) &
    (transactions_with_categ['categ'] == 1)
]

transactions_details_by_day_categ1 = transactions_oct_categ1.groupby([
    transactions_oct_categ1['date'].dt.month,
    transactions_oct_categ1['date'].dt.day
]).agg(
    number_of_transactions=('id_prod', 'count'),
    total_revenue=('price', 'sum')
).rename(index={10: 'October'}, level=1)

transactions_details_by_day_categ1

# Top 10 des clients

In [ ]:
transactions_by_client = data_filtered['client_id'].value_counts().head(10)

plt.figure(figsize=(10, 6))
transactions_by_client.plot(kind='bar', color='green')
plt.title('Top 10 des clients par nombre de transactions')
plt.xlabel('Identifiant du client')
plt.ylabel('Nombre de transactions')
plt.show()

transactions_by_client

# Top 10 des clients ayant le plus dépensé

In [ ]:
transactions_et_prix_df = pd.merge(transactions, products, on='id_prod', how='left')

montant_total_par_client = transactions_et_prix_df.groupby('client_id')['price'].sum().reset_index()

montant_total_par_client = montant_total_par_client.sort_values(by='price', ascending=False)

top_10_clients_par_depenses = montant_total_par_client.head(10)
top_10_clients_par_depenses


# Nettoyage des 4 gros clients

In [ ]:
clients_a_exclure = ['c_1609', 'c_4958', 'c_6714', 'c_3454']
donnees_sans_gros_clients = transactions_et_prix_df[~transactions_et_prix_df['client_id'].isin(clients_a_exclure)]

transactions_par_client_sans_gros = donnees_sans_gros_clients['client_id'].value_counts().reset_index()
transactions_par_client_sans_gros.columns = ['client_id', 'nombre_transactions']

montant_total_par_client_sans_gros = donnees_sans_gros_clients.groupby('client_id')['price'].sum().reset_index().sort_values(by='price', ascending=False)

top_10_clients_par_transactions_sans_gros = transactions_par_client_sans_gros.head(10)
top_10_clients_par_depenses_sans_gros = montant_total_par_client_sans_gros.head(10)

top_10_clients_par_transactions_sans_gros, top_10_clients_par_depenses_sans_gros

# Top 10 des clients suite au nettoyage

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(18, 6))

axes[0].bar(top_10_clients_par_transactions_sans_gros['client_id'], top_10_clients_par_transactions_sans_gros['nombre_transactions'], color='skyblue')
axes[0].set_title('Top 10 des clients par nombre de transactions')
axes[0].set_xlabel('Client ID')
axes[0].set_ylabel('Nombre de transactions')
axes[0].tick_params(axis='x')

axes[1].bar(top_10_clients_par_depenses_sans_gros['client_id'], top_10_clients_par_depenses_sans_gros['price'], color='lightgreen')
axes[1].set_title('Top 10 des clients par montant total dépensé')
axes[1].set_xlabel('Client ID')
axes[1].set_ylabel('Montant total dépensé (€)')
axes[1].tick_params(axis='x')

plt.tight_layout()
plt.show()

# Clients sans transactions

In [ ]:
nombre_de_clients_sans_achat = 0
clients_sans_achat = set(customers['client_id']) - set(transactions['client_id'])
for client in clients_sans_achat:
    nombre_de_clients_sans_achat += 1
print("Nombre de clients n'ayant pas effectué d'achat :", nombre_de_clients_sans_achat)

clients_sans_achat_df = pd.DataFrame(list(clients_sans_achat), columns=['client_id'])
chemin_csv = 'data/clients_sans_achat.csv'
clients_sans_achat_df.to_csv(chemin_csv, index=False)

# En prenant la variable nettoyée

In [ ]:
nombre_de_clients_sans_test = 0
clients_sans_test = set(transactions['date'].str.startswith('test'))
for client in clients_sans_test:
    nombre_de_clients_sans_test += 1
print("Nombre de clients n'ayant pas effectué d'achat :", nombre_de_clients_sans_test)

In [ ]:
nombre_de_clients_sans_achat_clean = 0
clients_sans_achat_clean = set(customers['client_id']) - set(transactions_cleaned['client_id'])

print("Noms des clients n'ayant pas effectué d'achat :")
for client_id in clients_sans_achat_clean:
    nombre_de_clients_sans_achat_clean += 1
    client_name = customers.loc[customers['client_id'] == client_id, 'client_id'].iloc[0]
    print(f"- {client_name}")

print("Nombre de clients n'ayant pas effectué d'achat :", nombre_de_clients_sans_achat_clean)

# Médiane des livres de toutes les catégories

In [ ]:
median_prix_tous_livres = products['price'].median()
print("Médiane des prix de tous les livres:", median_prix_tous_livres)

# Changement du produit sans prix

In [ ]:
nouveau_produit = pd.DataFrame([{'id_prod': '0_2245', 'price': median_prix_tous_livres, 'categ': pd.NA}])
products_df_modifie = pd.concat([products, nouveau_produit], ignore_index=True)
produit_0_2245_ajoute = products_df_modifie[products_df_modifie['id_prod'] == '0_2245']
produit_0_2245_ajoute
products_df_modifie.loc[products_df_modifie['id_prod'] == '0_2245', 'categ'] = 0
produit_0_2245_mis_a_jour = products_df_modifie[products_df_modifie['id_prod'] == '0_2245']
produit_0_2245_mis_a_jour

# CA par catégorie

In [ ]:
ca_par_categorie = transactions_et_prix_df.groupby('categ')['price'].sum()

ca_par_categorie

In [ ]:
fig, ax = plt.subplots()
ca_par_categorie.plot(kind='bar', ax=ax)
ax.set_title('Chiffre d\'affaires par catégorie de produit')
ax.set_xlabel('Catégorie de produit')
ax.set_ylabel('Chiffre d\'affaires (€)')
plt.show()

# Transactions par genre et catégorie

In [ ]:
transactions_clients_df = pd.merge(transactions_et_prix_df, customers[['client_id', 'sex']], on='client_id', how='left')

achats_par_categorie_et_genre = transactions_clients_df.groupby(['categ', 'sex']).size().unstack(fill_value=0)

achats_par_categorie_et_genre

# 1 ligne par client

In [ ]:
current_year = pd.Timestamp('now').year
customers['age'] = current_year - customers['birth']

merged_data = transactions_cleaned.merge(customers, on='client_id')
merged_data = merged_data.merge(products, on='id_prod')

clients_agg = merged_data.groupby(['client_id', 'sex', 'age']).agg(
    frequency=('session_id', pd.Series.nunique),
    total_revenue=('price', 'sum')
).reset_index()

clients_agg['average_basket'] = clients_agg['total_revenue'] / clients_agg['frequency']

categ_revenue = merged_data.pivot_table(
    index='client_id',
    columns='categ',
    values='price',
    aggfunc='sum',
    fill_value=0
)

final_table = clients_agg.join(categ_revenue, on='client_id')

final_table

chemin_fichier_csv_table = 'data/one_by_client.csv'
final_table.to_csv(chemin_fichier_csv_table, index=False)

chemin_fichier_csv_table

# Corrélation

In [ ]:
correlation_final = final_table[['age', 'frequency', 'total_revenue', 'average_basket', 0, 1, 2]].corr()
correlation_final

# HeatMap

In [ ]:
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_final, annot=True, fmt=".2f", cmap='coolwarm')
plt.title('Correlation')
plt.show()